In [1]:
import hopsworks
import pandas as pd
import joblib
from PIL import Image
import datetime
import requests

/home/jdowling/anaconda3/envs/book/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
project = hopsworks.login()
fs = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://snurran.hops.works:443/p/5240
Connected. Call `.close()` to terminate connection gracefully.


In [3]:
mr = project.get_model_registry()
model = mr.get_model("titanic", version=1)
model_dir = model.download()
model = joblib.load(model_dir + "/titanic_model.pkl")

External IP not configured for the Istio ingress gateway, the Hopsworks client will be used for model inference instead
Connected. Call `.close()` to terminate connection gracefully.


In [4]:
feature_view = fs.get_feature_view(name="titanic", version=1)
feature_view.init_batch_scoring(training_dataset_version=1)
batch_data = feature_view.get_batch_data()
batch_data

Finished: Reading data from Hopsworks, using Hive (2.50s) 


sex        age  pclass       fare  parch  sibsp  embarked
0      0  28.000000       2   13.00000      0      0         0
1      1  18.000000       2   23.00000      1      0         0
2      1  29.699118       3    7.22920      0      0         1
3      0  22.000000       1  135.63330      0      0         1
4      0   4.000000       3   31.27500      2      4         0
..   ...        ...     ...        ...    ...    ...       ...
760    1  30.000000       1  106.42500      0      0         1
761    0  29.000000       2   27.72080      0      1         1
762    0  18.000000       3    7.77500      0      0         0
763    1  45.000000       3    7.75000      0      0         0
764    1  20.563611       3  107.13882      1      1         1

[765 rows x 7 columns]

In [5]:
y_pred = model.predict(batch_data)
passenger_survived = y_pred[y_pred.size-1]
passenger_survived

0

In [6]:
passenger_details = batch_data.iloc[-1]
def remove_last_line_from_string(s):
    return s[:s.rfind('\n')]
passenger_details = remove_last_line_from_string(str(passenger_details))

with open("../../docs/_includes/titanic.html", "w", newline="\n") as file:
    file.write(passenger_details)

In [7]:
img_url = "https://raw.githubusercontent.com/featurestorebook/mlfs-book/main/docs/titanic/assets/img/titanic_" + str(passenger_survived) + ".jpg"
img = Image.open(requests.get(img_url, stream=True).raw)            
img.save("../../docs/titanic/assets/img/latest_titanic_survival.jpg")